In [1]:
using PyPlot

INFO: Loading help data...


In [1]:
type Maze
    definition::String
    special_values::Dict{Char, Int64}
    string_states::Matrix{Char}
    maze::Matrix{Int64}
    neighbours::Dict{Int64, Vector{Int64}}
    neighbour_positions::Dict{Int64, Vector{(Int64,Int64)}}
    values::Vector{Int64}
    policy::Vector{Symbol}
end

In [2]:
maze_def="
#######
#T    #
#     #
#     #
#     #
#    T#
#######
"

state_values = ['T' => 0, ' ' => -1]

Dict{Char,Int64} with 2 entries:
  ' ' => -1
  'T' => 0

In [35]:
function convert_string_to_string_array(s::String)
    index = 1
    while s[index] == ' ' || s[index] == '\n'
        index = index + 1
    end
    maze = nothing
    row = []
    for i=index:length(s)
        if s[i] == '\n'
            if maze == nothing
                maze = row
            else
                maze = hcat(maze, row)
            end
            row = []
        else
            row = [row, s[i]]
        end
    end
    return maze
end

function name_states(maze::Matrix{Char})
    states = zeros(size(maze))
    state_name = 1
    for i=1:size(maze)[1]
        for j=1:size(maze)[2]
            if maze[i,j] == '#'
                states[i,j] = 0
            elseif maze[i,j] == ' '
                states[i,j] = state_name
                state_name = state_name + 1
            else
                states[i,j] = -1    
            end
        end
    end
    int64(states)
end

initialise_value_function(maze::Matrix{Int64}) = zeros(maximum(maze))

function greedy_policy!(m::Maze)
    maze   = m.maze
    values = m.values
    m.policy = ["" for i=1:length(values)]
    for i = 2:size(maze)[1]-1
        for j = 2:size(maze)[2]-1
            if maze[i,j] > 0
                possible_actions = []
                action_values = []
                if maze[i,j-1] > 0
                    possible_actions = [possible_actions, :north]
                    action_values    = [action_values, values[maze[i,j-1]]]
                end
                if maze[i,j+1] > 0
                    possible_actions = [possible_actions, :south]
                    action_values    = [action_values, values[maze[i,j+1]]]
                end

                if maze[i-1,j] > 0
                    possible_actions = [possible_actions, :west]
                    action_values    = [action_values, values[maze[i-1,j]]]
                end
                if maze[i+1,j] > 0
                    possible_actions = [possible_actions, :east]
                    action_values    = [action_values, values[maze[i+1,j]]]
                end
                best_action_value = maximum(action_values)
                possible_actions = possible_actions[findin(action_values,best_action_value)]
                action = possible_actions[int(ceil(rand() * length(possible_actions)))]
                m.policy[maze[i,j]] = action
            end
        end
    end
end

function execute_action(action::Symbol)
    p = rand()
    if p < 0.9
        return action
    end
    p = rand()
    if action == :north
        if p < 0.5
            return :west
        else
            return :east
        end
    elseif action == :east
        if p < 0.5
            return :north
        else
            return :south
        end
    elseif action == :south
        if p < 0.5
            return :west
        else
            return :east
        end
    elseif action == :east
        if p < 0.5
            return :north
        else
            return :south
        end    
    end
    return nothing
end

function get_neighbours(named_states::Matrix{Int64})
    neighbours = [-1=>[2,3]]
    neighbour_positions = [-1 => [(2,2)]]
    for i = 2:size(named_states)[1]-1
        for j = 2:size(named_states)[2]-1
            state_neighbours = []
            state_neighbour_positions = [(0,0)]
            state_name = named_states[i,j]
            if named_states[i-1,j] != 0
                state_neighbours = [state_neighbours, named_states[i-1,j]]
                state_neighbour_positions = [state_neighbour_positions, (i-1, j)]
            end
            if named_states[i+1,j] != 0
                state_neighbours = [state_neighbours, named_states[i+1,j]]
                state_neighbour_positions = [state_neighbour_positions, (i+1, j)]
            end
            if named_states[i,j-1] != 0
                state_neighbours = [state_neighbours, named_states[i,j-1]]
                state_neighbour_positions = [state_neighbour_positions, (i, j-1)]
            end
            if named_states[i,j+1] != 0
                state_neighbours = [state_neighbours, named_states[i,j+1]]
                state_neighbour_positions = [state_neighbour_positions, (i, j+1)]
            end
            neighbours[state_name] = state_neighbours
            neighbour_positions[state_name] = state_neighbour_positions[2:end]
        end
    end
    pop!(neighbours, -1)
    pop!(neighbour_positions, -1)
    neighbours, neighbour_positions
end

string_states = convert_string_to_string_array(maze_def)
named_states = name_states(string_states)
get_neighbours(named_states)

function create_maze_element(maze::String, state_values::Dict{Char, Int64})
    string_states = convert_string_to_string_array(maze)
    named_states = name_states(string_states)
    values = initialise_value_function(named_states)
    neighbours, neighbour_positions = get_neighbours(named_states)
    m = Maze(maze, state_values, string_states, named_states, neighbours, neighbour_positions, values, [], )
    greedy_policy!(m)
    m
end



update_value_function! (generic function with 1 method)

In [36]:
maze = create_maze_element(maze_def, state_values)
#maze_state_names = name_states(maze_array)

Maze("\n#######\n#T    #\n#     #\n#     #\n#     #\n#    T#\n#######\n",[' '=>-1,'T'=>0],7x7 Array{Char,2}:
 '#'  '#'  '#'  '#'  '#'  '#'  '#'
 '#'  'T'  ' '  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  ' '  '#'
 '#'  ' '  ' '  ' '  ' '  'T'  '#'
 '#'  '#'  '#'  '#'  '#'  '#'  '#',7x7 Array{Int64,2}:
 0   0   0   0   0   0  0
 0  -1   1   2   3   4  0
 0   5   6   7   8   9  0
 0  10  11  12  13  14  0
 0  15  16  17  18  19  0
 0  20  21  22  23  -1  0
 0   0   0   0   0   0  0,[18=>[13,23,17,19],2=>[7,1,3],16=>[11,21,15,17],11=>[6,16,10,12],21=>[16,20,22],7=>[2,12,6,8],9=>[4,14,8],10=>[5,15,11],19=>[14,-1,18],17=>[12,22,16,18],8=>[3,13,7,9],22=>[17,21,23],6=>[1,11,5,7],4=>[9,3],3=>[8,2,4],5=>[-1,10,6],20=>[15,21],23=>[18,22,-1],13=>[8,18,12,14],14=>[9,19,13],15=>[10,20,16],12=>[7,17,11,13],1=>[6,-1,2]],[18=>[(4,5),(6,5),(5,4),(5,6)],2=>[(3,4),(2,3),(2,5)],16=>[(4,3),(6,3),(5,2),(5,4)],11=>[(3,3),(5,3),(4,2),(4,4)

In [10]:
greedy_policy!(maze)

In [63]:
function update_value_function!(m::Maze)
    values = zeros(length(m.values))
    for s = 1:length(m.values)
        new_value = 0
        for neighbour_index = 1:length(m.neighbours[s])
            neighbour          = m.neighbours[s][neighbour_index]
            neighbour_position = m.neighbour_positions[s][neighbour_index]
            if neighbour > 0
                new_value = new_value + m.values[neighbour] - 1.0
            else
                tag = m.string_states[neighbour_position[1],neighbour_position[2]]
                new_value = new_value + m.special_values[tag] - 1.0
            end
        end
        values[s] = new_value
        delta = maximum(abs(m.values - values))
        m.values = [v for v in values]
    end
end
maze = create_maze_element(maze_def, state_values)

for i = 1:10
update_value_function!(maze)
println(maze.values)
end

[-3,-6,-9,-11,-3,-10,-20,-33,-47,-6,-20,-44,-81,-131,-9,-33,-81,-166,-300,-11,-47,-131,-300]
[-19,-22,-25,-27,-3,-26,-52,-81,-111,-6,-36,-92,-177,-291,-9,-49,-145,-326,-620,-11,-63,-211,-540]
[-51,-54,-57,-59,-3,-58,-116,-177,-239,-6,-68,-188,-369,-611,-9,-81,-273,-646,-1260,-11,-95,-371,-1020]
[-115,-118,-121,-123,-3,-122,-244,-369,-495,-6,-132,-380,-753,-1251,-9,-145,-529,-1286,-2540,-11,-159,-691,-1980]
[-243,-246,-249,-251,-3,-250,-500,-753,-1007,-6,-260,-764,-1521,-2531,-9,-273,-1041,-2566,-5100,-11,-287,-1331,-3900]
[-499,-502,-505,-507,-3,-506,-1012,-1521,-2031,-6,-516,-1532,-3057,-5091,-9,-529,-2065,-5126,-10220,-11,-543,-2611,-7740]
[-1011,-1014,-1017,-1019,-3,-1018,-2036,-3057,-4079,-6,-1028,-3068,-6129,-10211,-9,-1041,-4113,-10246,-20460,-11,-1055,-5171,-15420]
[-2035,-2038,-2041,-2043,-3,-2042,-4084,-6129,-8175,-6,-2052,-6140,-12273,-20451,-9,-2065,-8209,-20486,-40940,-11,-2079,-10291,-30780]
[-4083,-4086,-4089,-4091,-3,-4090,-8180,-12273,-16367,-6,-4100,-12284,-24561,-4093

In [32]:
a[1]=>"a"

LoadError: unsupported or misplaced expression =>
while loading In[32], in expression starting on line 1